In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas numpy faiss-cpu sentence-transformers transformers
!pip install transformers accelerate bitsandbytes
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [4]:
link = "/content/drive/MyDrive/Negotiation/FLAT_RCL.txt"  #Put location of FLAT_RCL.txt here
df = pd.read_csv(link , delimiter="\t", header=None, on_bad_lines='skip')
df.head()

<ipython-input-4-bc7b3719ef0d>:2: DtypeWarning: Columns (17,18,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(link , delimiter="\t", header=None, on_bad_lines='skip')


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215021000000202,NaN,NaN,NaN
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215022000000202,NaN,NaN,NaN
2,3,02V236000,JAYCO,FT EAGLE 10 SG,2003,NaN,EQUIPMENT:OTHER:LABELS,"JAYCO, INC.",20020730.0,20020813.0,...,NaN,NaN,"ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...","IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...",OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...,"ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",000015210000106403000000349,NaN,NaN,NaN
3,4,02V237000,HOLIDAY RAMBLER,ENDEAVOR,2000,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000083965000000272,NaN,NaN,NaN
4,5,02V237000,HOLIDAY RAMBLER,ENDEAVOR,1999,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000080938000000272,NaN,NaN,NaN


In [5]:
data = df[df[2].str.lower().isin(['ford', 'toyota'])]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215021000000202,NaN,NaN,NaN
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719.0,20010531.0,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215022000000202,NaN,NaN,NaN
148,149,02V249000,FORD,CROWN VICTORIA,2002,02S39,"FUEL SYSTEM, OTHER:STORAGE:TANK ASSEMBLY",FORD MOTOR COMPANY,20010510.0,20020322.0,...,NaN,NaN,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ...",ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015251000215009000000162,NaN,NaN,NaN
300,301,02V239000,FORD,NAVIGATOR,2000,02L10,EQUIPMENT:OTHER:LABELS,FORD MOTOR COMPANY,19990331.0,20000806.0,...,571.0,120.0,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015220000089746000000349,NaN,NaN,NaN
301,302,02V239000,FORD,EXPEDITION,2000,02L10,EQUIPMENT:OTHER:LABELS,FORD MOTOR COMPANY,19990331.0,20000806.0,...,571.0,120.0,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015220000203219000000349,NaN,NaN,NaN


In [6]:
data = data[[2, 3, 4, 19, 20, 21]]
data.columns = ['make', 'model', 'year', 'defect summary', 'consequence summary', 'corrective summary']
data.head()

,make,model,year,defect summary,consequence summary,corrective summary
0,FORD,FOCUS,2000,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
1,FORD,FOCUS,2001,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
148,FORD,CROWN VICTORIA,2002,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ..."
300,FORD,NAVIGATOR,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...
301,FORD,EXPEDITION,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...


In [7]:
data = data.reset_index(drop=True)
data.head()

,make,model,year,defect summary,consequence summary,corrective summary
0,FORD,FOCUS,2000,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
1,FORD,FOCUS,2001,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
2,FORD,CROWN VICTORIA,2002,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ..."
3,FORD,NAVIGATOR,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...
4,FORD,EXPEDITION,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...


In [8]:
data = data.reset_index(drop=True)
data.head()

,make,model,year,defect summary,consequence summary,corrective summary
0,FORD,FOCUS,2000,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
1,FORD,FOCUS,2001,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
2,FORD,CROWN VICTORIA,2002,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ..."
3,FORD,NAVIGATOR,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...
4,FORD,EXPEDITION,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...


In [9]:
data = data.fillna('')

In [10]:
data['document'] = data['defect summary'] + ' ' + data['consequence summary'] + ' ' + data['corrective summary']
data.head()

,make,model,year,defect summary,consequence summary,corrective summary,document
0,FORD,FOCUS,2000,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
1,FORD,FOCUS,2001,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
2,FORD,CROWN VICTORIA,2002,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ...","ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT..."
3,FORD,NAVIGATOR,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...
4,FORD,EXPEDITION,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...


In [11]:
from huggingface_hub import login

login("Put HuggingFace Token Here")


In [12]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
documents = data['document'].tolist()
document_embeddings = embedding_model.encode(documents, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
document_embeddings = document_embeddings.cpu().detach().numpy()


In [14]:
dimension = document_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings)

In [15]:
def retrieve_documents(user_input, data, embedding_model, faiss_index, top_k=5):
    query = f"{user_input['make']} {user_input['model']} {user_input['year']} {user_input['issue']}"
    query_embedding = embedding_model.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding.cpu().detach().numpy()
    distances, indices = faiss_index.search(query_embedding, top_k)
    retrieved_docs = data.iloc[indices[0]]
    return retrieved_docs

In [16]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B",use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B",
    load_in_8bit=True,
    use_auth_token=True,
    device_map="auto",
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [20]:
def generate_summary(retrieved_docs, user_input):
    context = ' '.join(retrieved_docs['document'].tolist())

    prompt = f"Summarize the following text in brief paragraph \n\nContext:\n{context}\n\nSummary:"

    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model.generate(**inputs, max_new_tokens=200, num_beams=5, no_repeat_ngram_size=3)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

In [18]:
def summarization_agent(user_input, data, embedding_model, faiss_index):
    retrieved_docs = retrieve_documents(user_input, data, embedding_model, faiss_index, top_k=5)

    summary = generate_summary(retrieved_docs, user_input)

    final_response = {
        'retrieved_documents': retrieved_docs.to_dict(orient='records'),
        'summary': summary
    }
    return final_response

In [21]:
user_input = {
    'make': 'ford',
    'model': 'escape',
    'year': '2001',
    'issue': 'stuck throttle risk'
}

response = summarization_agent(user_input, data, embedding_model, faiss_index)

print("Retrieved Documents:")
for doc in response['retrieved_documents']:
    print(doc)
    print("\n---\n")

print("Summary:")
print(response['summary'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved Documents:
{'make': 'FORD', 'model': 'ESCAPE', 'year': 2001, 'defect summary': 'Ford Motor Company is recalling certain model year 2001 through 2004 Escape vehicles equipped with 3.0L V6 engines and speed control manufactured from October 22, 1999, through January 23, 2004.  Inadequate clearance between the engine cover and the speed control cable connector could result in a stuck throttle when the accelerator pedal is fully or almost-fully depressed.  This risk exists regardless of whether or not speed control (cruise control) is used.', 'consequence summary': 'A stuck throttle may result in very high vehicle speeds and make it difficult to stop or slow the vehicle, which could cause a crash, serious injury or death. ', 'corrective summary': "Ford will notify owners, and dealers will repair the vehicles by increasing the engine cover clearance, free of charge.  The safety recall began August 3, 2012.  Remedy parts are expected to be available in mid-August 2012.  Until then 